In [15]:
!pip install openai-whisper

In [13]:
import torch
print(torch.__version__)              # ตรวจสอบเวอร์ชัน PyTorch
print(torch.cuda.is_available())      # ตรวจสอบการรองรับ GPU
print(torch.version.cuda)             # ตรวจสอบเวอร์ชัน CUDA ที่ใช้งานได้
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

2.5.1
True
12.1
cuda


In [14]:
import whisper

model = whisper.load_model("base", device="cuda")

# Transcribe the .wav file
result = model.transcribe("./demo/paeall.wav")

# Print the transcription result
print(result["text"])
print(model.device)

c:\Users\ThanakornNemo\anaconda3\envs\voicecraft\lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_loc

 I love Thai culture. I would like to try cooking Thai food myself. Could you let comment a good restaurant for me? What do you like to do in your free time? Which Thai dish is your absolute favorite?
cuda:0


In [15]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["USER"] = "me" # TODO change this to your username

import torch
import torchaudio
import numpy as np
import random
from argparse import Namespace

from data.tokenizer import (
    AudioTokenizer,
    TextTokenizer,
)
from huggingface_hub import hf_hub_download

import shutil

device = "cpu"
voicecraft_name="830M_TTSEnhanced.pth"

from models import voicecraft
model = voicecraft.VoiceCraft.from_pretrained(f"pyp1/VoiceCraft_{voicecraft_name.replace('.pth', '')}")
phn2num = model.args.phn2num
config = vars(model.args)
model.to(device)

print(f'device is {device}')

encodec_fn = "./pretrained_models/encodec_4cb2048_giga.th"
if not os.path.exists(encodec_fn):
    print("Downloading encodec model...")
    os.system(f"wget https://huggingface.co/pyp1/VoiceCraft/resolve/main/encodec_4cb2048_giga.th")
    shutil.move("encodec_4cb2048_giga.th", "./pretrained_models/encodec_4cb2048_giga.th") # for windows
    
audio_tokenizer = AudioTokenizer(signature=encodec_fn, device=device) # will also put the neural codec model on gpu

text_tokenizer = TextTokenizer(backend="espeak")


device is cpu


c:\windows\system32\src\audiocraft\audiocraft\utils\checkpoint.py:93: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(checkpoint_path, 'cpu')
c:\Users\Thana

In [ ]:
orig_audio = "./demo/paeall.wav"

import whisper
whisper_model = whisper.load_model("base", device="cuda")
result = whisper_model.transcribe(orig_audio)
print(f'whisper text: {result["text"]}')
print(f'whisper device is {whisper_model.device}')

orig_transcript = result["text"]
input_text = " There was a man named Paetong who loved solstice very much. He thought about solstice every day and dreamed about solstice every night."

# move the audio and transcript to temp folder
temp_folder = "./demo/temp"
os.makedirs(temp_folder, exist_ok=True)
shutil.copy(orig_audio, temp_folder)
filename = os.path.splitext(orig_audio.split("/")[-1])[0]
with open(f"{temp_folder}/{filename}.txt", "w") as f:
    f.write(orig_transcript)

# run MFA to get the alignment
align_temp = f"{temp_folder}/mfa_alignments"
!source ~/.bashrc && \
    conda activate voicecraft && \
    mfa align -v --clean -j 1 --output_format csv {temp_folder} \
        english_us_arpa english_us_arpa {align_temp}

whisper text:  I love Thai culture. I would like to try cooking Thai food myself. Could you let comment a good restaurant for me? What do you like to do in your free time? Which Thai dish is your absolute favorite?
whisper device is cuda:0


'source' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
target_transcript = orig_transcript + input_text
audio_fn = f"{temp_folder}/{filename}.wav"
info = torchaudio.info(audio_fn)
audio_dur = info.num_frames / info.sample_rate
cut_off_sec = audio_dur - 0.001
prompt_end_frame = int(cut_off_sec * info.sample_rate)
codec_audio_sr = 16000
codec_sr = 50
top_k = 0
top_p = 0.9 # can also try 0.8, but 0.9 seems to work better
temperature = 1
silence_tokens=[1388,1898,131]
kvcache = 1 # NOTE if OOM, change this to 0, or try the 330M model

# NOTE adjust the below three arguments if the generation is not as good
stop_repetition = 3 # NOTE if the model generate long silence, reduce the stop_repetition to 3, 2 or even 1
sample_batch_size = 3 # NOTE: if the if there are long silence or unnaturally strecthed words, increase sample_batch_size to 4 or higher. What this will do to the model is that the model will run sample_batch_size examples of the same audio, and pick the one that's the shortest. So if the speech rate of the generated is too fast change it to a smaller number.
seed = 1 # change seed if you are still unhappy with the result

def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
seed_everything(seed)

decode_config = {'top_k': top_k, 'top_p': top_p, 'temperature': temperature, 'stop_repetition': stop_repetition, 'kvcache': kvcache, "codec_audio_sr": codec_audio_sr, "codec_sr": codec_sr, "silence_tokens": silence_tokens, "sample_batch_size": sample_batch_size}
from inference_tts_scale import inference_one_sample
concated_audio, gen_audio = inference_one_sample(model, Namespace(**config), phn2num, text_tokenizer, audio_tokenizer, audio_fn, target_transcript, device, decode_config, prompt_end_frame)
        
# save segments for comparison
concated_audio, gen_audio = concated_audio[0].cpu(), gen_audio[0].cpu()

# display the audio
from IPython.display import Audio
print("concatenate prompt and generated:")
display(Audio(concated_audio, rate=codec_audio_sr))
print("generated:")
display(Audio(gen_audio, rate=codec_audio_sr))